In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab040-other-sdks"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='49df6e10-10b1-4f8a-8217-9561dcac40c5', created_at=datetime.datetime(2025, 7, 28, 4, 2, 24, 337495, tzinfo=TzInfo(UTC)), name='54be2c13-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 4, 2, 24, 337495, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='27df566e-6c92-4f86-a7fc-a2b6a1f28018', agent_id='cd4b9256-144b-4aef-949b-f9000995013b', sequence_id=252, task_id='49df6e10-10b1-4f8a-8217-9561dcac40c5', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 4, 2, 24, 776038, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭───────────────────────── USER [07/28/2025 04:02:24] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:02:35] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news about AI and AI startups",                          │
│    "model": "gpt-4o-mini",                                                   │
│    "type": "web_search_preview"                                              │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 04:02:35] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "As of July 28, 2025, the artificial intelligence (AI) sector     │
│  continues to experience significant growth and innovation. Here are some o  │
│  the latest developments:\n\n**Surge in AI Startup Funding**\n\nIn the firs  │
│  half of 2025, U.S. startup funding increased by 75.6%, reaching $162.8      │
│  billion\u2014the second-highest on record. This surge is largely driven by  │
│  AI investments, which constituted 64.1% of total deal value during this     │
│  period. Major tech firms have been pivotal in this trend, with OpenAI       │
│  securing a $40 billion funding round and Meta investing $14.3 billion in    │
│  Scale AI.                                                                   │
│  ([reuters.com](https://www.reuters.com/business/us-ai-startups-see-funding  │
│  urge-while-more-vc-funds-struggle-raise-data-shows-2025-07-15/?utm_source=  │
│  enai))\n\n**Legal Disputes in AI Development**\n\nTech startup iyO Inc. ha  │
│  filed a lawsuit against former executive Dan Sargent, alleging the leak of  │
│  confidential product designs. This follows a previous trademark infringeme  │
│  lawsuit against OpenAI and designer Jony Ive over a similar-sounding AI     │
│  interface venture. The legal battles underscore the competitive and         │
│  secretive nature of AI development.                                         │
│  ([apnews.com](https://apnews.com/article/0193532cf71cf975de62b4218ebd3bb6?  │
│  m_source=openai))\n\n**Meta's Potential Investment in Scale AI**\n\nMeta    │
│  Platforms is reportedly in discussions to invest over $10 billion in Scale  │
│  AI, a data labeling startup. This potential investment highlights the       │
│  growing importance of data infrastructure in AI development. Scale AI,      │
│  founded in 2016, has previously received backing from major tech companies  │
│  such as Nvidia and Amazon.                                                  │
│  ([reuters.com](https://www.reuters.com/business/meta-talks-scale-ai-invest  │
│  nt-that-could-top-10-billion-bloomberg-news-reports-2025-06-08/?utm_source  │
│  penai))\n\n**Emerald AI's Energy-Efficient Data Centers**\n\nNvidia,        │
│  alongside Radical Ventures and AMPLO, has invested in Emerald AI, a startu  │
│  aiming to revolutionize data center energy use. Emerald AI's software alig  │
│  AI computation loads with regional electricity grid demands, allowing data  │
│  centers to support energy infrastructure rather than burden it. A recent    │
│  field test demonstrated the software's ability to dynamically shift         │
│  workloads in real time.                                                     │
│  ([axios.com](https://www.axios.com/2025/07/01/nvidia-startup-data-center-p  │
│  er?utm_source=openai))\n\n**Thinking Machines Lab's Rapid                   │
│  Growth**\n\nFounded in February 2025 by Mira Murati, former CTO of OpenAI,  │
│  Thinking Machines Lab has quickly gained traction. By July, the company     │
│  secured a $2 billion early-stage funding round led by Andreessen Horowitz,  │
│  valuing it at $12 billion. The startup has attracted talent from            │
│  competitors, including OpenAI co-founder John Schulman.                     │
│  ([en.wikipedia.org](https://en.wikipedia.org/wiki/Thinking_Machines_Lab?ut  │
│  source=openai))\n\n**Base44's

╭──────────────────────── AGENT [07/28/2025 04:02:35] ─────────────────────────╮
│ Here's the latest news on AI and AI startups as of July 28, 2025:            │
│                                                                              │
│  1 Surge in AI Startup Funding: U.S. startup funding has increased           │
│    significantly, with AI investments representing a major portion. This     │
│    includes a massive $40 billion round for OpenAI and $14.3 billion from    │
│    Meta for Scale AI. ]8;id=137415;https://www.reuters.com/business/us-ai-startups-see-funding-surge-while-more-vc-funds-struggle-raise-data-shows-2025-07-15/?utm_source=openai\Read more]8;;\.                                             │
│  2 Legal Disputes in AI Development: Tech startup iyO Inc. is in legal       │
│    battles regarding leaked product designs and trademark issues with OpenAI │
│    and Jony Ive. ]8;id=245135;https://apnews.com/article/0193532cf71cf975de62b4218ebd3bb6?utm_source=openai\Learn more]8;;\.                                                 │
│  3 Meta's Investment in Scale AI: Meta is considering a $10 billion          │
│    investment in Scale AI, underlining the importance of data infrastructure │
│    in AI. ]8;id=369642;https://www.reuters.com/business/meta-talks-scale-ai-investment-that-could-top-10-billion-bloomberg-news-reports-2025-06-08/?utm_source=openai\Details here]8;;\.                                                      │
│  4 Emerald AI's Energy-Efficient Solutions: Nvidia and other investors are   │
│    backing Emerald AI to improve data center energy use in alignment with    │
│    electricity grid demands. ]8;id=234177;https://www.axios.com/2025/07/01/nvidia-startup-data-center-power?utm_source=openai\Find out more]8;;\.                                  │
│  5 Thinking Machines Lab's Rapid Growth: Founded by Mira Murati, this        │
│    startup quickly drew $2 billion in funding, attracting talent from major  │
│    competitors. ]8;id=709301;https://en.wikipedia.org/wiki/Thinking_Machines_Lab?utm_source=openai\Read more about it]8;;\.                                          │
│  6 Base44's Acquisition by Wix: Base44, known for its AI-powered platform    │
│    for creating applications through conversational interfaces, was acquired │
│    by Wix for about $80 million. ]8;id=806967;https://en.wikipedia.org/wiki/Base44?utm_source=openai\More information]8;;\.                           │
│  7 Harvey's Series D Funding: Legal AI firm Harvey raised $300 million, led  │
│    by Sequoia Capital, to advance its AI capabilities for legal processes.   │
│    ]8;id=247028;https://en.wikipedia.org/wiki/Harvey_%28software%29?utm_source=openai\Explore further]8;;\.                                                          │
│                                                                              │
│ These highlights demonstrate the vibrant and rapidly evolving nature of AI   │
│ startups, with substantial investments, strategic developments, and          │
│ innovative applications across various sectors.                              │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
